# Генерация синтетических данных на основе выданных
Сюда приходит файл __X_train_clothes.csv__, содержащий отзывы только из категории "одежда"
Остальные файлы используются лишь для как различного рода проверок (__X_train.csv__)

In [6]:
!pip uninstall -y torch torchvision torchaudio fastai
!pip install -q "transformers==4.41.2" "datasets==2.19.1" "accelerate==0.30.1" "bitsandbytes==0.43.1" "torch==2.3.0"

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: fastai 2.7.19
Uninstalling fastai-2.7.19:
  Successfully uninstalled fastai-2.7.19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 66.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2

## Подготовка к генерации (Загрузка данных и модели)

In [23]:
X_train_clothes = pd.read_csv('/kaggle/input/traint-bank-sirius/X_train_clothes.csv')

In [24]:
import pandas as pd
categories = [
    'бытовая техника',
    'обувь',
    'одежда',
    'посуда',
    'текстиль',
    'товары для детей',
    'украшения и аксессуары',
    'электроника',
    'нет товара'
]

X_train = pd.read_csv('/kaggle/input/traint-bank-sirius/X_train.csv')
y_train = pd.read_csv('/kaggle/input/traint-bank-sirius/y_true.csv', header=None, names=['target'])
df_qwen_v1 = pd.read_csv('/kaggle/input/traint-bank-sirius/qwen2.5-7B_instruct_df_train_v1.csv')
y_qwen_v2 = pd.read_csv('/kaggle/input/traint-bank-sirius/predictions_qwen_v2.csv')
# df_qwen_v1['pred']

In [25]:
X_train_full = X_train.copy()
X_train_full['target'] = y_train
X_train_full['pred_qwen_v2'] = y_qwen_v2
X_train_full[(X_train_full['target'] == 'одежда') & (X_train_full['pred_qwen_v2'] == 'одежда')]
X_train_full['text'].to_csv('/kaggle/working/X_train_clothes_v1.csv')

In [27]:
mask_neq = (y_train['target'] != df_qwen_v1['pred'])
X_train_all = X_train[mask_neq].copy()
X_train_all['target'] = y_train[mask_neq]['target']
X_train_all['pred'] = df_qwen_v1[mask_neq]['pred']
pd.set_option('display.max_rows', False)
X_train_all.to_csv('X_train_all.csv')

In [28]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "Qwen/Qwen2.5-7B-Instruct"

# Задаем конфиг для 4-битной квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

#загружаем модель с квантизацией
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)
print("модель и токенизатор загружены")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

модель и токенизатор загружены


## Генерация данных на основе выданных

#### Как вообще будет происходить генерация.
Просмотрев датасет, я выяснил у него есть только два достаточно полных класса - "нет товара" и "одежда". Очевидно, что класс "нет товара" существенно отличается от других краткостью, неинформативностью отзывов, поэтому отзывы из этой категории будут нерелевантны для генерации отзывов других категорий ('бытовая техника','обувь', 'посуда','текстиль', 'товары для детей','украшения и аксессуары','электроника').

А вот класс "одежда" отлично подойдет для этой задачи. Поэтому мы будем генерировать данные, отталкиваясь только от этого класса.

Генерировать будем следующим образом.
Параметры генерации.
1) Есть название товара
   * В каком месте название товара в отзыве (начало/середина/конец)
2) Есть только описание товара
   * В каком месте описание товара в отзыве (начало/середина/конец)
По хорошему нужно было помимо разметки данных, сохранять информации о размышлении модели(на основании чего была выбрана та или иная категория (название товара, описание)), чтобы для первого типа генерации подавать примеры с таким же типом из другого класса, для второго типа - ... .Но что уже поделать, не знал, что это потребуется, а время на это - нет(((

Как устроен промпт

на вход __build_prompt__ подаваться
*   Категория для которой нужно сгенерировать пример.
*   Название самого товара (`product_name`).
*   Тип генерации (`generation_type`): 'название' или 'описание'.
*   Позиция в тексте (`position`): 'начало', 'середина', 'конец'.
*   **Тональность отзыва (`sentiment`): 'позитивный', 'негативный'.**
*   **Пол автора (`gender`): 'мужчина', 'женщина'.**
*   10 примеров из категории 'одежда' (`examples`).
*   Часть описания выбора категории (`classifier_prompt_part`).

Это описание я писал давно, так что возможно стоит взглянуть на код

In [29]:
# Список названий товаров для категорий для которых нужно сгенерировать данные
PRODUCT_NAME_STORAGE = {
    "товары для детей": [
        "игрушка", "конструктор", "кукла", "машинка", "коляска", "соска", "подгузник",
        "кроватка", "автокресло", "самокат", "пинетки", "боди", "ползунки", "детский комбинезон",
        "стульчик для кормления", "манеж", "погремушка", "бутылочка для кормления", "слюнявчик",
        "развивающий коврик", "мобиль на кроватку", "ходунки", "прыгунки", "велосипед трехколесный",
        "беговел", "детский шампунь", "влажные салфетки", "ванночка для купания", "горшок",
        "радионяня", "видеоняня", "сортер", "пирамидка", "детские пазлы", "книжка-игрушка",
        "детское мыло", "термометр для воды", "пустышка", "прорезыватель для зубов", "царапки",
        "чепчик", "детский крем", "присыпка", "ниблер", "стерилизатор для бутылочек"
    ],
    "обувь": [
        "кроссовки", "кеды", "туфли", "ботинки", "сапоги", "босоножки", "сандалии",
        "шлепанцы", "тапочки", "ботильоны", "балетки", "угги", "мокасины", "слипоны",
        "лоферы", "эспадрильи", "сабо", "мюли", "ботфорты", "полусапоги", "резиновые сапоги",
        "дутики", "валенки", "унты", "бутсы", "чешки", "аквашузы", "ортопедическая обувь",
        "рабочие ботинки", "берцы", "гриндерсы", "оксфорды", "броги", "топсайдеры",
        "домашние тапочки", "вьетнамки", "сланцы", "кроксы", "хайтопы", "треккинговые ботинки"
    ],
    "украшения и аксессуары": [
        "сумка", "рюкзак", "клатч", "кошелек", "шапка", "шляпа", "кепка", "шарф", "платок",
        "перчатки", "ремень", "очки", "часы", "зонт", "кольцо", "серьги", "браслет", "цепочка",
        "колье", "брошь", "галстук", "панама", "берет", "повязка на голову", "бейсболка",
        "снуд", "палантин", "варежки", "митенки", "сумка-шоппер", "поясная сумка", "косметичка",
        "визитница", "ключница", "кулон", "подвеска", "запонки", "зажим для галстука",
        "пирсинг", "кафф", "заколка", "резинка для волос", "ободок", "галстук-бабочка",
        "портмоне", "обложка для паспорта", "бандана", "солнцезащитные очки", "кардхолдер"
    ],
    "текстиль": [
        "постельное белье", "простыня", "пододеяльник", "наволочка", "одеяло", "подушка",
        "плед", "покрывало", "полотенце", "скатерть", "шторы", "тюль", "чехол на диван",
        "наматрасник", "декоративная наволочка", "банный халат", "коврик для ванной",
        "кухонное полотенце", "прихватка", "фартук", "салфетка тканевая", "дорожка на стол",
        "чехол на кресло", "чехол на стул", "римская штора", "портьера", "полотенце для рук",
        "полотенце для ног", "пляжное полотенце", "гобелен", "флисовый плед", "махровая простыня",
        "ортопедическая подушка", "подушка для беременных", "одеяло из верблюжьей шерсти",
        "шелковое постельное белье", "льняная скатерть", "вафельное полотенце"
    ],
    "посуда": [
        "тарелка", "миска", "чашка", "кружка", "стакан", "бокал", "вилка", "ложка", "нож",
        "кастрюля", "сковорода", "противень", "форма для выпечки", "доска разделочная",
        "чайник (неэлектрический)", "блюдце", "салатник", "пиала", "супница", "блюдо для сервировки",
        "менажница", "соусник", "масленка", "сахарница", "солонка", "перечница", "салфетница",
        "поднос", "рюмка", "стопка", "фужер", "графин", "кувшин", "сотейник", "ковш", "дуршлаг",
        "сито", "половник", "шумовка", "венчик", "скалка", "терка", "овощечистка", "штопор",
        "открывалка", "мерный стакан", "контейнер для еды", "ланч-бокс", "термос", "френч-пресс"
    ],
    "бытовая техника": [
        "холодильник", "плита", "духовка", "микроволновка", "чайник электрический", "кофеварка",
        "миксер", "блендер", "тостер", "мультиварка", "стиральная машина", "пылесос", "утюг",
        "фен", "обогреватель", "вентилятор", "эпилятор", "электробритва", "посудомоечная машина",
        "морозильная камера", "кухонная вытяжка", "соковыжималка", "мясорубка", "кухонные весы",
        "йогуртница", "вафельница", "гриль электрический", "хлебопечка", "пароварка", "сушильная машина",
        "робот-пылесос", "пароочиститель", "отпариватель", "увлажнитель воздуха", "осушитель воздуха",
        "ионизатор", "швейная машина", "электрическая зубная щетка", "ирригатор", "массажер",
        "весы напольные", "машинка для стрижки волос", "триммер", "кондиционер", "тепловентилятор"
    ],
    "электроника": [
        "телефон", "смартфон", "планшет", "ноутбук", "наушники", "колонка", "телевизор",
        "монитор", "фотоаппарат", "зарядное устройство", "пауэрбанк", "флешка", "карта памяти",
        "кабель", "мышь", "клавиатура", "смарт-часы", "фитнес-браслет", "роутер", "веб-камера",
        "принтер", "сканер", "проектор", "внешний жесткий диск", "SSD-накопитель", "саундбар",
        "домашний кинотеатр", "видеокамера", "экшн-камера", "диктофон", "электронная книга",
        "игровая приставка", "геймпад", "джойстик", "игровой руль", "умная колонка", "умная розетка",
        "умная лампочка", "квадрокоптер", "дрон", "GPS-навигатор", "видеорегистратор",
        "Wi-Fi репитер", "графический планшет", "микрофон", "процессор", "видеокарта"
    ]
}

In [17]:
# Когда я делал разметку датасета X_train.csv у меня сформировалась инструкция, которая 
# теперь потребуется для генерации отзывов миноритарных классов

#Благодаря такому разбиению промпт не будет содержать информацию о всех классах, 
# что очень удобно, потому что для больших промптов результаты могут быть плачевными)


PROMPT_PARTS = {
    # Этот блок будет в каждом запросе.
    "general_rules": """
--- ГЛАВНЫЕ ПРИНЦИПЫ КЛАССИФИКАЦИИ (для использования на ШАГЕ 1)--- 
**ПРИНЦИП №1: УПОМИНАНИЕ ТОВАРА ВАЖНЕЕ ВСЕГО.**
Если в отзыве прямо назван конкретный товар (например, 'трусы', 'платье', 'телефон'), ты ОБЯЗАН выбрать категорию этого товара, даже если вся остальная часть отзыва посвящена проблемам с доставкой или спором.
-   **КЛЮЧЕВОЙ ПРИМЕР:** "Трусы не пришли"
-   **РАЗМЫШЛЕНИЯ:** Товар 'трусы' четко назван. 'Трусы' относятся к категории 'одежда'. Жалоба "не пришли" касается доставки, но так как товар назван, мы игнорируем проблему доставки и классифицируем по товару.
-   **РЕШЕНИЕ:** одежда

--- АЛГОРИТМ ПРИНЯТИЯ РЕШЕНИЯ ---
**ШАГ 1: ПРОВЕРКА НА ПРЯМОЕ УПОМИНАНИЕ ТОВАРА (ЗОЛОТОЕ ПРАВИЛО).**
Сначала найди в отзыве название конкретного товара.
-   **ЕСЛИ** в отзыве назван товар (например, 'рюкзак', 'майка', 'трусы', 'платье'), **ТО** немедленно определи его категорию и **ПОЛНОСТЬЮ ПРОИГНОРИРУЙ** все остальные слова про доставку, споры или деньги.

**ШАГ 2: ПРОВЕРКА НА ФИЗИЧЕСКОЕ ОПИСАНИЕ.**
-   **ЕСЛИ** на ШАГЕ 1 товар не был найден, проверь, есть ли в отзыве физическое описание или дефект ('низ рваный', 'ткань колется', 'не включается').
-   **ЕСЛИ** описание есть, определи категорию по нему.

**ШАГ 3: КАТЕГОРИЯ "НЕТ ТОВАРА".**
-   **ЕСЛИ** ШАГ 1 и ШАГ 2 НЕ ПРОЙДЕНЫ, **ТОЛЬКО ТОГДА** выбери категорию **'нет товара'**.

--- ГЛАВНЫЕ ПРИНЦИПЫ КЛАССИФИКАЦИИ (для использования на ШАГЕ 2)--- 
**ПРИНЦИП №1: ФИЗИЧЕСКОЕ ОПИСАНИЕ ВАЖНЕЕ ЖАЛОБ НА СЕРВИС.**
Если товар не назван, но есть его физическое описание или дефект ('низ рваный', 'ткань колется', 'не включается'), классифицируй по этому описанию. Жалобы на споры или деньги в этом случае игнорируются.
-   **ПРИМЕР:** "Низ рваный..деньги не вернули! Открыла спор." -> РЕШЕНИЕ: одежда
-   **[НЕГАТИВНЫЙ ПРИМЕР]** "Ужасная синтетика" -> РЕШЕНИЕ: нет товара
""",

    # отсюда берется только необходимая категория
    "category_specific_rules": {
        "товары для детей": """**КАТЕГОРИЯ: товары для детей (ВЫСШИЙ ПРИОРИТЕТ)**
**ПРАВИЛО:** Любой товар, если он явно предназначен для ребенка.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** игрушка, конструктор, кукла, 'сыну'/'дочке', 'ребенок доволен'.
-   **Пример:** "Куртка для сына оказалась очень теплой."
-   **РЕШЕНИЕ:** товары для детей.""",

        "нет товара": """**КАТЕГОРИЯ: нет товара**
**ПРАВИЛО:** Выбирай эту категорию ТОЛЬКО если из отзыва абсолютно невозможно понять, о каком товаре идет речь.
**СЮДА ОТНОСИТСЯ:** Отзывы только о доставке, упаковке, продавце, возврате денег.
-   **Пример:** "Шло очень долго, но упаковано хорошо."
-   **РЕШЕНИЕ:** нет товара.""",

        "одежда": """**КАТЕГОРИЯ: одежда**
**ПРАВИЛО:** Вещи, которые надевают на тело. ИСКЛЮЧАЕТ обувь и аксессуары.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** платье, юбка, брюки, джинсы, кофта; размер (S, M, 44), 'село идеально', 'маломерит', 'рукава короткие'.
-   **Пример:** "Кофта хорошая, теплая, но после стирки вся в катышках."
-   **РЕШЕНИЕ:** одежда.""",

        "обувь": """**КАТЕГОРИЯ: обувь**
**ПРАВИЛО:** Все, что имеет подошву и надевается на стопу.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** кроссовки, туфли, ботинки; размер (39, 42), 'жмут', 'натирают пятку', 'удобная колодка'.
-   **Пример:** "На свой 38 взяла 39, и все равно жмут. Очень узкие."
-   **РЕШЕНИЕ:** обувь.""",

        "украшения и аксессуары": """**КАТЕГОРИЯ: украшения и аксессуары**
**ПРАВИЛО:** Предметы для украшения и дополнения образа. **Сюда относятся шапки, шарфы, перчатки, сумки, ремни.**
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** серьги, кольцо, браслет, сумка; 'застежка', 'камень выпал', 'потемнело', 'молния сломалась'.
-   **Пример:** "Сумка красивая, но фурнитура очень дешевая, вся облезла."
-   **РЕШЕНИЕ:** украшения и аксессуары.""",
        
        "текстиль": """**КАТЕГОРИЯ: текстиль**
**ПРАВИЛО:** Тканевые изделия для дома и интерьера. Не одежда.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** постельное белье, простыня, полотенце, шторы; 'мягкое', 'впитывает воду', 'после стирки село', 'полиняло'.
-   **Пример:** "Полотенца ужасные, жесткие и совсем не впитывают воду."
-   **РЕШЕНИЕ:** текстиль.""",

        "посуда": """**КАТЕГОРИЯ: посуда**
**ПРАВИЛО:** Предметы для готовки и еды, не являющиеся электроприборами.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** кастрюля, сковорода, тарелка, чашка; 'пригорает', 'ручка греется', 'скол', 'треснула'.
-   **Пример:** "На сковородке все пригорает, покрытие облезло."
-   **РЕШЕНИЕ:** посуда.""",

        "бытовая техника": """**КАТЕГОРИЯ: бытовая техника**
**ПРАВИЛО:** Электрические приборы для дома (не персональные гаджеты).
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** пылесос, утюг, чайник, миксер; 'не работает', 'шумит', 'греется', 'мощность', 'кнопка'.
-   **Пример:** "Чайник воняет пластиком, даже после десятого кипячения."
-   **РЕШЕНИЕ:** бытовая техника.""",

        "электроника": """**КАТЕГОРИЯ: электроника**
**ПРАВИЛО:** Персональные цифровые устройства, гаджеты и комплектующие.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** телефон, наушники, планшет, смарт-часы; 'батарея', 'экран', 'звук', 'bluetooth', 'не подключается', 'зависает'.
-   **Пример:** "Беспроводные наушники постоянно теряют связь с телефоном."
-   **РЕШЕНИЕ:** электроника."""
    }
}

In [56]:
import pandas as pd
import json
from typing import List, Dict
from itertools import cycle
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm


def get_model_response(prompt: str, model, tokenizer, device: str) -> str:
    messages = [{"role": "system", "content": "You are a helpful assistant that generates data in JSON format."}, {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=1024, do_sample=True, temperature=0.6, top_p=0.9, pad_token_id=tokenizer.eos_token_id)
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    json_start_index = full_response.find('{')
    if json_start_index != -1:
        potential_json = full_response[json_start_index:]
        json_end_index = potential_json.rfind('}')
        if json_end_index != -1:
            return potential_json[:json_end_index + 1]
        return potential_json
    else:
        print(f"\nПредупреждение: JSON не найден в ответе модели. Ответ был:\n{full_response}")
        return '{"reviews": []}'

def build_generation_prompt(product_name: str, target_category: str, style_examples_df: pd.DataFrame, general_rules: str, category_specific_rules: str) -> str:
    examples_list = style_examples_df['text'].tolist()
    examples_str = "\n".join([f"- {ex}" for ex in examples_list])
    focused_instructions = f"{general_rules}\n\n{category_specific_rules}"
    master_prompt = f"""
Твоя задача — создать набор реалистичных отзывов для товара.

--- ТЕХНИЧЕСКОЕ ЗАДАНИЕ (ПРАВИЛА КЛАССИФИКАЦИИ) ---
Внимательно изучи эти правила. Твои сгенерированные отзывы должны стать идеальными примерами для этих правил.
{focused_instructions}

--- ПРИМЕРЫ СТИЛЯ ---
Твои отзывы должны быть похожи на них по стилю.
{examples_str}

--- ОСНОВНАЯ ЗАДАЧА: СИСТЕМАТИЧЕСКАЯ ГЕНЕРАЦИЯ ---
Для товара "{product_name}" (категория "{target_category}") сгенерируй набор отзывов, покрывающих следующие случаи:
1.  Позитивный отзыв с явным упоминанием товара.
2.  Негативный отзыв с явным упоминанием товара и жалобой на качество.
3.  Отзыв с упоминанием товара, но основной жалобой на сервис (доставка/возврат).
4.  Отзыв с физическим описанием дефекта, но без упоминания названия товара.
5.  Отзыв, который должен быть классифицирован как 'нет товара' (только про доставку/упаковку).

--- ВЫХОДНОЙ ФОРМАТ ---
Верни результат в виде JSON ОБЪЕКТА с ключом "reviews" (список объектов с ключами "text" и "target").
"""
    return master_prompt
def generate_for_one_product(product_name: str, target_category: str, full_clothing_df: pd.DataFrame, prompt_parts: Dict, model, tokenizer, device: str) -> pd.DataFrame:
    # ... (эта функция остается без изменений)
    sample_size = min(15, len(full_clothing_df))
    sampled_examples_df = full_clothing_df.sample(n=sample_size)
    general_rules = prompt_parts['general_rules']
    category_specific_rules = prompt_parts['category_specific_rules'][target_category]
    final_prompt = build_generation_prompt(product_name=product_name, target_category=target_category, style_examples_df=sampled_examples_df, general_rules=general_rules, category_specific_rules=category_specific_rules)
    llm_response_str = get_model_response(final_prompt, model, tokenizer, device)
    try:
        data = json.loads(llm_response_str)
        review_list = data["reviews"]
        if not review_list:
             return pd.DataFrame(columns=['text', 'target'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"\nОшибка обработки ответа модели для '{product_name}': {e}")
        return pd.DataFrame(columns=['text', 'target'])
    return pd.DataFrame(review_list)

#Генерация датасета
def generate_dataset(
    target_total_count: int,
    full_clothing_df: pd.DataFrame,
    categories_to_generate: List[str],
    product_storage: Dict,
    prompt_parts: Dict,
    model,
    tokenizer,
    device: str
) -> pd.DataFrame:
    """
    Главная функция с прогресс-баром tqdm для отслеживания генерации.
    """
    all_generated_dfs = []
    
    product_cycle_list = []
    for category in categories_to_generate:
        for product in product_storage.get(category, []):
            product_cycle_list.append((product, category))

    if not product_cycle_list:
        print("Ошибка: нет товаров для генерации.")
        return pd.DataFrame()

    product_iterator = cycle(product_cycle_list)
    pbar = tqdm(total=target_total_count, desc="Генерация отзывов")
    
    total_generated_count = 0
    while total_generated_count < target_total_count:
        current_product, current_category = next(product_iterator)
        
        pbar.set_description(f"Товар: {current_product} ({current_category})")
        
        product_df = generate_for_one_product(
            product_name=current_product,
            target_category=current_category,
            full_clothing_df=full_clothing_df,
            prompt_parts=prompt_parts,
            model=model,
            tokenizer=tokenizer,
            device=device
        )
        
        if not product_df.empty:
            all_generated_dfs.append(product_df)
            generated_now = len(product_df)
            total_generated_count += generated_now
            
            pbar.update(generated_now)
    pbar.close()
    
    final_df = pd.concat(all_generated_dfs, ignore_index=True)
    final_df = final_df.head(target_total_count)
    
    print(f"\n===== Генерация завершена. Итоговый размер датасета: {len(final_df)} =====")
    return final_df

device = "cuda" if torch.cuda.is_available() else "cpu"

In [52]:
def get_model_response(prompt: str, model, tokenizer, device: str) -> str:
    """
    Отправляет промпт в модель и возвращает очищенный текстовый ответ.
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant that generates data in JSON format."},
        {"role": "user", "content": prompt}
    ]
    inputs = tokenizer.apply_chat_template(
        messages, 
        tokenize=True, 
        add_generation_prompt=True, 
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    json_start_index = full_response.find('{')
    
    if json_start_index != -1:
        potential_json = full_response[json_start_index:]
        json_end_index = potential_json.rfind('}')
        if json_end_index != -1:
            return potential_json[:json_end_index + 1]
        return potential_json # Возвращаем то, что нашли
    else:
        return '{"reviews": []}'



#### Генерация отзывов из категории __посуда__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["посуда"]
dataset_dishes = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_dishes)

Цель: сгенерировать 500 примеров.
Генерация для: 'тарелка' (посуда)... Добавлено 5 примеров. Всего: 5/500
Генерация для: 'миска' (посуда)... Добавлено 5 примеров. Всего: 10/500
Генерация для: 'чашка' (посуда)... Добавлено 5 примеров. Всего: 15/500
Генерация для: 'кружка' (посуда)... Добавлено 5 примеров. Всего: 20/500
Генерация для: 'стакан' (посуда)... Добавлено 5 примеров. Всего: 25/500
Генерация для: 'бокал' (посуда)... Добавлено 5 примеров. Всего: 30/500
Генерация для: 'вилка' (посуда)... Добавлено 5 примеров. Всего: 35/500
Генерация для: 'ложка' (посуда)... 
Ошибка обработки ответа модели для 'ложка': Expecting ',' delimiter: line 21 column 13 (char 791)
Добавлено 0 примеров. Всего: 35/500
Генерация для: 'нож' (посуда)... Добавлено 5 примеров. Всего: 40/500
Генерация для: 'кастрюля' (посуда)... Добавлено 5 примеров. Всего: 45/500
Генерация для: 'сковорода' (посуда)... Добавлено 5 примеров. Всего: 50/500
Генерация для: 'противень' (посуда)... 
Ошибка обработки ответа модели для 'пр

In [44]:
dataset_dishes = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_dishes.csv')
dataset_dishes = dataset_dishes[['text', 'target']]
dataset_dishes

,text,target
0,"Тарелка просто отличная! Качественная посуда, ...",посуда
1,"Купила тарелку для ежедневного использования, ...",посуда
2,"Заказал тарелку и получил ее вовремя, но пришл...",посуда
3,"На тарелке заметны трещины, хотя на сайте было...",посуда
4,Последний раз заказывал товары на Алиэкспресс ...,нет товара
5,Миска просто идеальная для моих котят! Они оче...,посуда
6,Миска постоянно ломается. На второй день испол...,посуда
7,Миска не пришла уже неделю. Заказал ее еще в н...,посуда
8,"На миске после стирки образовались катушки, хо...",посуда
...,...,...


In [43]:
dataset_dishes.to_csv('/kaggle/working/dataset_dishes.csv')

#### Генерация отзывов из категории __бытовая техника__

In [60]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["бытовая техника"]
dataset_house_apps = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_house_apps)



Генерация отзывов:   0%|          | 0/500 [00:00<?, ?it/s]

Товар: холодильник (бытовая техника):   0%|          | 0/500 [00:00<?, ?it/s]

Товар: холодильник (бытовая техника):   1%|          | 5/500 [00:19<32:22,  3.92s/it]

Товар: плита (бытовая техника):   1%|          | 5/500 [00:19<32:22,  3.92s/it]      

Товар: плита (бытовая техника):   2%|▏         | 10/500 [00:42<35:08,  4.30s/it]

Товар: духовка (бытовая техника):   2%|▏         | 10/500 [00:42<35:08,  4.30s/it]

Товар: духовка (бытовая техника):   3%|▎         | 15/500 [01:05<35:49,  4.43s/it]

Товар: микроволновка (бытовая техника):   3%|▎         | 15/500 [01:05<35:49,  4.43s/it]

Товар: микроволновка (бытовая техника):   4%|▍         | 20/500 [01:30<37:20,  4.67s/it]

Товар: чайник электрический (бытовая техника):   4%|▍         | 20/500 [01:30<37:20,  4.67s/it]

Товар: чайник электрический (бытовая техника):   5%|▌         | 25/500 [01:53<36:39,  4.63s/it]

Товар: кофеварка (бытовая техника):   5%|▌         | 25/500 [


Ошибка обработки ответа модели для 'тостер': Expecting ',' delimiter: line 21 column 13 (char 865)




Товар: мультиварка (бытовая техника):   9%|▉         | 45/500 [04:01<48:44,  6.43s/it]

Товар: стиральная машина (бытовая техника):   9%|▉         | 45/500 [04:01<48:44,  6.43s/it]

Товар: стиральная машина (бытовая техника):  10%|█         | 50/500 [04:32<47:55,  6.39s/it]

Товар: пылесос (бытовая техника):  10%|█         | 50/500 [04:32<47:55,  6.39s/it]          

Товар: пылесос (бытовая техника):  11%|█         | 55/500 [04:58<44:31,  6.00s/it]

Товар: утюг (бытовая техника):  11%|█         | 55/500 [04:58<44:31,  6.00s/it]   

Товар: утюг (бытовая техника):  12%|█▏        | 60/500 [05:24<42:04,  5.74s/it]

Товар: фен (бытовая техника):  12%|█▏        | 60/500 [05:24<42:04,  5.74s/it] 

Товар: фен (бытовая техника):  13%|█▎        | 65/500 [05:47<39:01,  5.38s/it]

Товар: обогреватель (бытовая техника):  13%|█▎        | 65/500 [05:47<39:01,  5.38s/it]

Товар: вентилятор (бытовая техника):  13%|█▎        | 65/500 [05:58<39:01,  5.38s/it]  


Ошибка обработки ответа модели для 'обогреватель': Expecting ',' delimiter: line 10 column 6 (char 323)




Товар: вентилятор (бытовая техника):  14%|█▍        | 70/500 [06:20<41:14,  5.76s/it]

Товар: эпилятор (бытовая техника):  14%|█▍        | 70/500 [06:20<41:14,  5.76s/it]  

Товар: эпилятор (бытовая техника):  15%|█▌        | 75/500 [06:47<40:19,  5.69s/it]

Товар: электробритва (бытовая техника):  15%|█▌        | 75/500 [06:47<40:19,  5.69s/it]

Товар: электробритва (бытовая техника):  16%|█▌        | 80/500 [07:09<37:06,  5.30s/it]

Товар: посудомоечная машина (бытовая техника):  16%|█▌        | 80/500 [07:09<37:06,  5.30s/it]

Товар: посудомоечная машина (бытовая техника):  17%|█▋        | 85/500 [07:33<35:23,  5.12s/it]

Товар: морозильная камера (бытовая техника):  17%|█▋        | 85/500 [07:33<35:23,  5.12s/it]  

Товар: морозильная камера (бытовая техника):  18%|█▊        | 90/500 [08:05<37:33,  5.50s/it]

Товар: кухонная вытяжка (бытовая техника):  18%|█▊        | 90/500 [08:05<37:33,  5.50s/it]  

Товар: кухонная вытяжка (бытовая техника):  19%|█▉        | 95/500 [08:29<35:4

...


Ошибка обработки ответа модели для 'хлебопечка': Invalid control character at: line 20 column 87 (char 1009)




Товар: пароварка (бытовая техника):  67%|██████▋   | 335/500 [31:06<21:28,  7.81s/it]

Товар: сушильная машина (бытовая техника):  67%|██████▋   | 335/500 [31:06<21:28,  7.81s/it]

Товар: сушильная машина (бытовая техника):  68%|██████▊   | 340/500 [31:36<19:26,  7.29s/it]

Товар: робот-пылесос (бытовая техника):  68%|██████▊   | 340/500 [31:36<19:26,  7.29s/it]   

Товар: робот-пылесос (бытовая техника):  69%|██████▉   | 345/500 [32:03<17:21,  6.72s/it]

Товар: пароочиститель (бытовая техника):  69%|██████▉   | 345/500 [32:03<17:21,  6.72s/it]

Товар: пароочиститель (бытовая техника):  70%|███████   | 350/500 [32:31<15:53,  6.35s/it]

Товар: отпариватель (бытовая техника):  70%|███████   | 350/500 [32:31<15:53,  6.35s/it]  

Товар: отпариватель (бытовая техника):  71%|███████   | 355/500 [32:59<14:48,  6.13s/it]

Товар: увлажнитель воздуха (бытовая техника):  71%|███████   | 355/500 [32:59<14:48,  6.13s/it]

Товар: увлажнитель воздуха (бытовая техника):  72%|███████▏  | 360/500 [33:


Ошибка обработки ответа модели для 'плита': Invalid control character at: line 4 column 86 (char 114)




Товар: духовка (бытовая техника):  85%|████████▌ | 425/500 [39:05<07:37,  6.11s/it]

Товар: микроволновка (бытовая техника):  85%|████████▌ | 425/500 [39:05<07:37,  6.11s/it]

Товар: микроволновка (бытовая техника):  86%|████████▌ | 430/500 [39:32<06:50,  5.87s/it]

Товар: чайник электрический (бытовая техника):  86%|████████▌ | 430/500 [39:32<06:50,  5.87s/it]

Товар: чайник электрический (бытовая техника):  87%|████████▋ | 435/500 [40:02<06:23,  5.91s/it]

Товар: кофеварка (бытовая техника):  87%|████████▋ | 435/500 [40:02<06:23,  5.91s/it]           

Товар: кофеварка (бытовая техника):  88%|████████▊ | 440/500 [40:24<05:30,  5.50s/it]

Товар: миксер (бытовая техника):  88%|████████▊ | 440/500 [40:24<05:30,  5.50s/it]   

Товар: миксер (бытовая техника):  89%|████████▉ | 445/500 [40:48<04:50,  5.28s/it]

Товар: блендер (бытовая техника):  89%|████████▉ | 445/500 [40:48<04:50,  5.28s/it]

Товар: блендер (бытовая техника):  90%|█████████ | 450/500 [41:11<04:14,  5.08s/it]

Товар: то


===== Генерация завершена. Итоговый размер датасета: 500 =====
                                                  text           target
0    Холодильник отлично работает, удобный и вмести...      холодильник
1    Приобрел холодильник, но он постоянно шумит и ...      холодильник
2    Холодильник не прибыл вовремя и сломался при д...      холодильник
3    Ткань колется и постоянно пачкается. Не понятн...       нет товара
4    Получил коробку, которая была повреждена и сод...       нет товара
5    Плитка отлично работает, нагревается быстро и ...  бытовая техника
6    Плитка постоянно выдает странный запах и греет...  бытовая техника
7    Плитка не пришла по срокам, хотя в описании бы...  бытовая техника
8    Плитка имеет ржавые пятна на поверхности и нек...  бытовая техника
..                                                 ...              ...
491  Эпилятор не работает вообще. После первого исп...  бытовая техника
492  Заказал эпилятор, но он так и не пришел. Деньг...       нет товара


In [68]:
dataset_house_apps = dataset_house_apps[['text', 'target']]

In [69]:
dataset_house_apps['target'] = dataset_house_apps['target'].apply(lambda x: x if x == "нет товара" else "бытовая техника")
dataset_house_apps

,text,target
0,"Холодильник отлично работает, удобный и вмести...",бытовая техника
1,"Приобрел холодильник, но он постоянно шумит и ...",бытовая техника
2,Холодильник не прибыл вовремя и сломался при д...,бытовая техника
3,Ткань колется и постоянно пачкается. Не понятн...,нет товара
4,"Получил коробку, которая была повреждена и сод...",нет товара
5,"Плитка отлично работает, нагревается быстро и ...",бытовая техника
6,Плитка постоянно выдает странный запах и греет...,бытовая техника
7,"Плитка не пришла по срокам, хотя в описании бы...",бытовая техника
8,Плитка имеет ржавые пятна на поверхности и нек...,бытовая техника
...,...,...


In [70]:
dataset_house_apps.to_csv('dataset_house_apps.csv')

In [71]:
!curl -T dataset_house_apps.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 136 537 bytes

wget https://bashupload.com/kgNcN/LRpRO.csv




#### Генерация отзывов из категории __электроника__

In [72]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["электроника"]
dataset_electronics = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_electronics)





Генерация отзывов:   0%|          | 0/500 [00:00<?, ?it/s]



Товар: телефон (электроника):   0%|          | 0/500 [00:00<?, ?it/s]



Товар: телефон (электроника):   1%|          | 5/500 [00:24<40:58,  4.97s/it]



Товар: смартфон (электроника):   1%|          | 5/500 [00:24<40:58,  4.97s/it]



Товар: смартфон (электроника):   2%|▏         | 10/500 [00:48<39:22,  4.82s/it]



Товар: планшет (электроника):   2%|▏         | 10/500 [00:48<39:22,  4.82s/it] 



Товар: планшет (электроника):   3%|▎         | 15/500 [01:09<36:39,  4.54s/it]



Товар: ноутбук (электроника):   3%|▎         | 15/500 [01:09<36:39,  4.54s/it]



Товар: наушники (электроника):   3%|▎         | 15/500 [01:56<36:39,  4.54s/it]


Ошибка обработки ответа модели для 'ноутбук': Invalid control character at: line 16 column 84 (char 833)






Товар: наушники (электроника):   4%|▍         | 20/500 [02:20<1:06:43,  8.34s/it]



Товар: колонка (электроника):   4%|▍         | 20/500 [02:20<1:06:43,  8.34s/it] 



Товар: колонка (электроника):   5%|▌         | 25/500 [02:48<58:10,  7.35s/it]  



Товар: телевизор (электроника):   5%|▌         | 25/500 [02:48<58:10,  7.35s/it]



Товар: телевизор (электроника):   6%|▌         | 30/500 [03:13<51:17,  6.55s/it]



Товар: монитор (электроника):   6%|▌         | 30/500 [03:13<51:17,  6.55s/it]  



Товар: монитор (электроника):   7%|▋         | 35/500 [03:35<45:22,  5.85s/it]



Товар: фотоаппарат (электроника):   7%|▋         | 35/500 [03:35<45:22,  5.85s/it]



Товар: фотоаппарат (электроника):   8%|▊         | 40/500 [03:57<41:33,  5.42s/it]



Товар: зарядное устройство (электроника):   8%|▊         | 40/500 [03:57<41:33,  5.42s/it]



Товар: зарядное устройство (электроника):   9%|▉         | 45/500 [04:20<39:09,  5.16s/it]



Товар: пауэрбанк (электроника):   9%|▉         |


Ошибка обработки ответа модели для 'карта памяти': Invalid control character at: line 16 column 94 (char 796)






Товар: кабель (электроника):  12%|█▏        | 60/500 [06:12<54:48,  7.47s/it]



Товар: мышь (электроника):  12%|█▏        | 60/500 [06:12<54:48,  7.47s/it]  



Товар: клавиатура (электроника):  12%|█▏        | 60/500 [06:28<54:48,  7.47s/it]

...


Ошибка обработки ответа модели для 'фотоаппарат': Invalid control character at: line 8 column 112 (char 298)






Товар: зарядное устройство (электроника):  95%|█████████▌| 475/500 [43:02<02:48,  6.74s/it]



Товар: пауэрбанк (электроника):  95%|█████████▌| 475/500 [43:02<02:48,  6.74s/it]          



Товар: пауэрбанк (электроника):  96%|█████████▌| 480/500 [43:26<02:02,  6.14s/it]



Товар: флешка (электроника):  96%|█████████▌| 480/500 [43:26<02:02,  6.14s/it]   



Товар: флешка (электроника):  97%|█████████▋| 485/500 [43:45<01:21,  5.46s/it]



Товар: карта памяти (электроника):  97%|█████████▋| 485/500 [43:45<01:21,  5.46s/it]



Товар: карта памяти (электроника):  98%|█████████▊| 490/500 [44:10<00:53,  5.32s/it]



Товар: кабель (электроника):  98%|█████████▊| 490/500 [44:10<00:53,  5.32s/it]      



Товар: кабель (электроника):  99%|█████████▉| 495/500 [44:30<00:24,  4.92s/it]



Товар: мышь (электроника):  99%|█████████▉| 495/500 [44:30<00:24,  4.92s/it]  



Товар: клавиатура (электроника):  99%|█████████▉| 495/500 [45:16<00:24,  4.92s/it]


Ошибка обработки ответа модели для 'мышь': Invalid control character at: line 12 column 107 (char 608)






Товар: клавиатура (электроника): 100%|██████████| 500/500 [45:35<00:00,  5.47s/it]


===== Генерация завершена. Итоговый размер датасета: 500 =====
                                                  text       target
0    Телефон работает отлично, очень быстрая камера...  электроника
1    Приобрел этот телефон недавно, но он постоянно...  электроника
2    Приобрел телефон, но заказ не дошел до адресат...  электроника
3    Телефон имеет небольшой скол на экране и зазор...  электроника
4    Отправили мне пакет вместо телефона. Доставили...   нет товара
5    Смартфон работает отлично! Батарея держит весь...  электроника
6    Купил смартфон, но качество его оставляет жела...  электроника
7    Долгая доставка смартфона вызывает разочарован...  электроника
8    Ткань чистая синтетика, топ пришел ужасно мяты...   нет товара
..                                                 ...          ...
491  Кабель постоянно вырывается из разъема и часто...  электроника
492  Кабель пришел только через неделю после заказа...   нет товара
493  Кабель слегка колется на ощупь, возможно, из с.

In [73]:
dataset_electronics = dataset_electronics[['text', 'target']]
dataset_electronics['target'] = dataset_electronics['target'].apply(lambda x: x if x == "нет товара" else "электроника")
dataset_electronics

,text,target
0,"Телефон работает отлично, очень быстрая камера...",электроника
1,"Приобрел этот телефон недавно, но он постоянно...",электроника
2,"Приобрел телефон, но заказ не дошел до адресат...",электроника
3,Телефон имеет небольшой скол на экране и зазор...,электроника
4,Отправили мне пакет вместо телефона. Доставили...,нет товара
5,Смартфон работает отлично! Батарея держит весь...,электроника
6,"Купил смартфон, но качество его оставляет жела...",электроника
7,Долгая доставка смартфона вызывает разочарован...,электроника
8,"Ткань чистая синтетика, топ пришел ужасно мяты...",нет товара
...,...,...


In [74]:
dataset_electronics.to_csv('dataset_electronics.csv')

In [75]:
!curl -T dataset_electronics.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 127 559 bytes

wget https://bashupload.com/EI9kq/Jy2x7.csv




#### Генерация отзывов из категории __текстиль__

In [76]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["текстиль"]
dataset_textile = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_textile)


Товар: подушка (текстиль):   4%|▍         | 20/500 [01:58<36:38,  4.58s/it]


Ошибка обработки ответа модели для 'одеяло': Expecting ',' delimiter: line 14 column 6 (char 544)



Товар: тюль (текстиль):   9%|▉         | 45/500 [04:33<37:01,  4.88s/it] 


Ошибка обработки ответа модели для 'шторы': Extra data: line 24 column 1 (char 937)



Товар: банный халат (текстиль):  12%|█▏        | 60/500 [06:21<45:19,  6.18s/it]          

...


Ошибка обработки ответа модели для 'шторы': Expecting ',' delimiter: line 6 column 1 (char 151)



Товар: фартук (текстиль):  83%|████████▎ | 415/500 [40:37<06:56,  4.90s/it]   


Ошибка обработки ответа модели для 'прихватка': Invalid control character at: line 4 column 61 (char 83)



Товар: шелковое постельное белье (текстиль): 100%|██████████| 500/500 [47:46<00:00,  5.73s/it]


===== Генерация завершена. Итоговый размер датасета: 500 =====
                                                  text            target
0    Купила постельное белье и осталась очень довол...  постельное белье
1    Постельное белье оказалось ужасным. Ткань жест...  постельное белье
2    Постельное белье не пришло и денег не вернули....  постельное белье
3    Постельное белье постирала много раз, но все р...  постельное белье
4    Постельное белье пришло в срок, но доставка за...        нет товара
5    Простыня оказалась очень удобной для сна. Она ...          текстиль
6    Приобрела эту простыню, но она оказалась очень...          текстиль
7    Ждала доставку больше месяца и только потом по...        нет товара
8    Постельное белье оказалось не соответствующим ...          текстиль
..                                                 ...               ...
491  Одеяло из верблюжьей шерсти оказалось очень же...           tekstil
492  Пришло одеяло из верблюжьей шерсти, но доставк...      

In [77]:
dataset_textile = dataset_textile[['text', 'target']]
dataset_textile['target'] = dataset_textile['target'].apply(lambda x: x if x == "нет товара" else "текстиль")
dataset_textile

,text,target
0,Купила постельное белье и осталась очень довол...,текстиль
1,Постельное белье оказалось ужасным. Ткань жест...,текстиль
2,Постельное белье не пришло и денег не вернули....,текстиль
3,"Постельное белье постирала много раз, но все р...",текстиль
4,"Постельное белье пришло в срок, но доставка за...",нет товара
5,Простыня оказалась очень удобной для сна. Она ...,текстиль
6,"Приобрела эту простыню, но она оказалась очень...",текстиль
7,Ждала доставку больше месяца и только потом по...,нет товара
8,Постельное белье оказалось не соответствующим ...,текстиль
...,...,...


In [79]:
dataset_textile.to_csv('dataset_textile.csv')

In [80]:
!curl -T dataset_textile.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 123 171 bytes

wget https://bashupload.com/d2wsE/p2L2F.csv




#### Генерация отзывов из категории __украшения и аксессуары__

In [81]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["украшения и аксессуары"]
dataset_decorations = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_decorations)


Товар: серьги (украшения и аксессуары):  14%|█▍        | 70/500 [06:11<35:03,  4.89s/it]


Ошибка обработки ответа модели для 'кольцо': Expecting ',' delimiter: line 6 column 10 (char 189)



Товар: брошь (украшения и аксессуары):  17%|█▋        | 85/500 [08:13<38:10,  5.52s/it]


Ошибка обработки ответа модели для 'колье': Invalid control character at: line 20 column 58 (char 909)



Товар: берет (украшения и аксессуары):  21%|██        | 105/500 [09:46<37:45,  5.73s/it]
Товар: повязка на голову (украшения и аксессуары):  21%|██        | 105/500 [09:46<37:45,  5.73s/it]
Товар: повязка на голову (украшения и аксессуары):  22%|██▏       | 110/500 [10:17<38:31,  5.93s/it]
Товар: запонки (украшения и аксессуары):  35%|███▌      | 175/500 [16:01<28:23,  5.24s/it]
Товар: зажим для галстука (украшения и аксессуары):  35%|███▌      | 175/500 [16:01<28:23,  5.24s/it]
Товар: зажим для галстука (украшения и аксессуары):  36%|███▌      | 180/500 [16:26<27:32,  5.16s/it]
Товар: заколка (украшения и аксессуары):  39%|███▉      | 195/500 [17:35<24:09,  4.75s/it]
Товар: резинка для волос (украшения и аксессуары):  39%|███▉      | 195/500 [17:35<24:09,  4.75s/it]
Товар: резинка для волос (украшения и аксессуары):  40%|████      | 200/500 [17:58<23:30,  4.70s/it]
Товар: портмоне (украшения и аксессуары):  43%|████▎     | 215/500 [19:06<21:21,  4.50s/it]
Товар: обложка для паспорта 

...


Ошибка обработки ответа модели для 'пирсинг': Invalid control character at: line 8 column 75 (char 279)



Товар: заколка (украшения и аксессуары):  84%|████████▍ | 420/500 [39:15<10:09,  7.62s/it]
Товар: резинка для волос (украшения и аксессуары):  84%|████████▍ | 420/500 [39:15<10:09,  7.62s/it]
Товар: резинка для волос (украшения и аксессуары):  85%|████████▌ | 425/500 [39:37<08:22,  6.69s/it]
Товар: портмоне (украшения и аксессуары):  88%|████████▊ | 440/500 [40:49<05:26,  5.43s/it]
Товар: обложка для паспорта (украшения и аксессуары):  88%|████████▊ | 440/500 [40:49<05:26,  5.43s/it]
Товар: обложка для паспорта (украшения и аксессуары):  89%|████████▉ | 445/500 [41:11<04:41,  5.12s/it]
Товар: бандана (украшения и аксессуары):  90%|█████████ | 450/500 [41:35<04:12,  5.04s/it]
Товар: солнцезащитные очки (украшения и аксессуары):  90%|█████████ | 450/500 [41:35<04:12,  5.04s/it]
Товар: солнцезащитные очки (украшения и аксессуары):  91%|█████████ | 455/500 [42:14<04:24,  5.87s/it]
Товар: кепка (украшения и аксессуары):  97%|█████████▋| 485/500 [44:50<01:13,  4.90s/it]


Ошибка обработки ответа модели для 'шляпа': Expecting ',' delimiter: line 10 column 10 (char 395)



Товар: платок (украшения и аксессуары): 100%|██████████| 500/500 [46:12<00:00,  5.55s/it]


===== Генерация завершена. Итоговый размер датасета: 500 =====
                                                  text                  target
0    Сумка просто великолепна! Качество отличное, у...  украшения и аксессуары
1    Сумка пришла в хорошем состоянии, но ткань ста...  украшения и аксессуары
2    Не получила заказ на следующий день, как обеща...              нет товара
3    Сумка была повреждена при доставке. Молния сло...  украшения и аксессуары
4    Посылка пришла быстро, но коробка была сильно ...              нет товара
5    Рюкзак отличный, удобный и вместительный! Каче...                  рюкзак
6    Купила этот рюкзак, но качество просто ужасное...                  рюкзак
7    Заказал рюкзак, но его не доставили уже две не...                  рюкзак
8    Обложка на рюкзаке начала рваться через пару д...              нет товара
..                                                 ...                     ...
491  Шарф очень некачественный. Ткань колется и быс...  украшения и

In [82]:
dataset_decorations = dataset_decorations[['text', 'target']]
dataset_decorations['target'] = dataset_decorations['target'].apply(lambda x: x if x == "нет товара" else "украшения и аксессуары")
dataset_decorations

,text,target
0,"Сумка просто великолепна! Качество отличное, у...",украшения и аксессуары
1,"Сумка пришла в хорошем состоянии, но ткань ста...",украшения и аксессуары
2,"Не получила заказ на следующий день, как обеща...",нет товара
3,Сумка была повреждена при доставке. Молния сло...,украшения и аксессуары
4,"Посылка пришла быстро, но коробка была сильно ...",нет товара
5,"Рюкзак отличный, удобный и вместительный! Каче...",украшения и аксессуары
6,"Купила этот рюкзак, но качество просто ужасное...",украшения и аксессуары
7,"Заказал рюкзак, но его не доставили уже две не...",украшения и аксессуары
8,Обложка на рюкзаке начала рваться через пару д...,нет товара
...,...,...


In [83]:
dataset_decorations.to_csv('dataset_decorations.csv')

In [84]:
!curl -T dataset_decorations.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 129 142 bytes

wget https://bashupload.com/c9kip/_14Ld.csv




#### Генерация отзывов из категории __обувь__

In [85]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["обувь"]
dataset_shoes = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_shoes)


Товар: кеды (обувь):   0%|          | 0/500 [00:42<?, ?it/s]     


Ошибка обработки ответа модели для 'кроссовки': Invalid control character at: line 12 column 107 (char 589)



Товар: ботинки (обувь):   1%|          | 5/500 [01:23<1:50:27, 13.39s/it]


Ошибка обработки ответа модели для 'туфли': Expecting ',' delimiter: line 14 column 10 (char 465)



Товар: босоножки (обувь):   2%|▏         | 10/500 [02:22<1:18:21,  9.59s/it]

...


Ошибка обработки ответа модели для 'туфли': Invalid control character at: line 16 column 90 (char 752)



Товар: лоферы (обувь):  98%|█████████▊| 490/500 [1:00:04<00:51,  5.19s/it]


Ошибка обработки ответа модели для 'слипоны': Expecting ',' delimiter: line 6 column 10 (char 192)



Товар: эспадрильи (обувь): 100%|██████████| 500/500 [1:00:50<00:00,  7.30s/it]


===== Генерация завершена. Итоговый размер датасета: 500 =====
                                                  text      target
0    Кеды просто супер! Очень удобные и стильные. Р...        кеды
1    Купила кеды по совету продавца, но они оказали...        кеды
2    Кеды не пришли до меня в срок. Заказ ждала мес...  нет товара
3    На кедах торчат нитки и даже есть дырка на одн...        кеды
4    Посылка пришла быстро, но кеды оказались очень...  нет товара
5    Ботинки очень удобные и хорошо сидят по ноге. ...     ботинки
6    Ботинки пришли в очень плохом состоянии, ткань...     ботинки
7    Заказал ботинки, но до сих пор их нет. Деньги ...  нет товара
8    Подошва начала отклеиваться после нескольких и...     ботинки
..                                                 ...         ...
491  Лоферы очень некачественные! Подошва начала ра...       обувь
492  Заказ не пришел. Обратилась в службу поддержки...  нет товара
493  Купила лоферы и заметила, что подошва немного ...       обув

In [86]:
dataset_shoes = dataset_shoes[['text', 'target']]
dataset_shoes['target'] = dataset_shoes['target'].apply(lambda x: x if x == "нет товара" else "обувь")
dataset_shoes

,text,target
0,Кеды просто супер! Очень удобные и стильные. Р...,обувь
1,"Купила кеды по совету продавца, но они оказали...",обувь
2,Кеды не пришли до меня в срок. Заказ ждала мес...,нет товара
3,На кедах торчат нитки и даже есть дырка на одн...,обувь
4,"Посылка пришла быстро, но кеды оказались очень...",нет товара
5,Ботинки очень удобные и хорошо сидят по ноге. ...,обувь
6,"Ботинки пришли в очень плохом состоянии, ткань...",обувь
7,"Заказал ботинки, но до сих пор их нет. Деньги ...",нет товара
8,Подошва начала отклеиваться после нескольких и...,обувь
...,...,...


In [87]:
dataset_shoes.to_csv('dataset_shoes.csv')

In [88]:
!curl -T dataset_shoes.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 123 110 bytes

wget https://bashupload.com/KVr0b/5LE_I.csv




#### Генерация отзывов из категории __товары для детей__

In [89]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["товары для детей"]
dataset_children = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_children)


Товар: подгузник (товары для детей):   5%|▌         | 25/500 [02:20<33:59,  4.29s/it]


Ошибка обработки ответа модели для 'соска': Invalid control character at: line 12 column 178 (char 659)



Товар: детский комбинезон (товары для детей):  13%|█▎        | 65/500 [05:33<38:12,  5.27s/it]
Товар: стульчик для кормления (товары для детей):  13%|█▎        | 65/500 [05:33<38:12,  5.27s/it]
Товар: стульчик для кормления (товары для детей):  14%|█▍        | 70/500 [06:07<41:03,  5.73s/it]
Товар: погремушка (товары для детей):  16%|█▌        | 80/500 [06:58<38:05,  5.44s/it]
Товар: бутылочка для кормления (товары для детей):  16%|█▌        | 80/500 [06:58<38:05,  5.44s/it]
Товар: бутылочка для кормления (товары для детей):  17%|█▋        | 85/500 [07:22<36:34,  5.29s/it]
Товар: ходунки (товары для детей):  19%|█▉        | 95/500 [09:01<37:17,  5.52s/it]           


Ошибка обработки ответа модели для 'мобиль на кроватку': Invalid control character at: line 16 column 209 (char 946)



Товар: прыгунки (товары для детей):  21%|██        | 105/500 [09:52<46:19,  7.04s/it]
Товар: велосипед трехколесный (товары для детей):  21%|██        | 105/500 [09:52<46:19,  7.04s/it]
Товар: велосипед трехколесный (товары для детей):  22%|██▏       | 110/500 [10:16<41:06,  6.33s/it]
Товар: детское мыло (товары для детей):  32%|███▏      | 160/500 [15:23<29:42,  5.24s/it]  

...


Ошибка обработки ответа модели для 'стульчик для кормления': Expecting ',' delimiter: line 18 column 10 (char 881)



Товар: погремушка (товары для детей):  99%|█████████▉| 495/500 [46:00<00:26,  5.30s/it]


Ошибка обработки ответа модели для 'манеж': Expecting ',' delimiter: line 18 column 10 (char 718)



Товар: погремушка (товары для детей): 100%|██████████| 500/500 [46:28<00:00,  5.58s/it]


===== Генерация завершена. Итоговый размер датасета: 500 =====
                                                  text              target
0    Игрушка просто замечательная! Дочке очень понр...             игрушка
1    Купила конструктор для сына, но качество остав...         конструктор
2    Заказала игрушку для дочки, но продавец отправ...             игрушка
3    Пришла кукла с дефектом – глаза отклеились. Ни...             игрушка
4    Отправили коробку вместо игрушки. Когда я позв...          нет товара
5    Конструктор просто отличный! Дети в восторге, ...         конструктор
6    Конструктор очень плохого качества, детали пос...         конструктор
7    Заказ ждала два месяца, но конструктор так и н...          нет товара
8    Пришел конструктор, но детали плохо склеиваютс...         конструктор
..                                                 ...                 ...
491  Комбинезон оказался очень маленьким для моей д...  детский комбинезон
492  Заказал комбинезон для дочери, 

In [90]:
dataset_children = dataset_children[['text', 'target']]
dataset_children['target'] = dataset_children['target'].apply(lambda x: x if x == "нет товара" else "товары для детей")
dataset_children

,text,target
0,Игрушка просто замечательная! Дочке очень понр...,товары для детей
1,"Купила конструктор для сына, но качество остав...",товары для детей
2,"Заказала игрушку для дочки, но продавец отправ...",товары для детей
3,Пришла кукла с дефектом – глаза отклеились. Ни...,товары для детей
4,Отправили коробку вместо игрушки. Когда я позв...,нет товара
5,"Конструктор просто отличный! Дети в восторге, ...",товары для детей
6,"Конструктор очень плохого качества, детали пос...",товары для детей
7,"Заказ ждала два месяца, но конструктор так и н...",нет товара
8,"Пришел конструктор, но детали плохо склеиваютс...",товары для детей
...,...,...


In [91]:
dataset_children.to_csv('dataset_children.csv')

In [92]:
!curl -T dataset_children.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 138 156 bytes

wget https://bashupload.com/Bc8FW/g5sME.csv




-----
----

In [10]:
X_train_pred = pd.read_csv('/kaggle/input/traint-bank-sirius/X_train_pred.csv')
X_train_pred = X_train_pred[['text', 'target']]
X_train_pred

,text,target
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",нет товара
1,"футболка хорошего качества,но футболка не как ...",одежда
2,Все отлично!!!,нет товара
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",одежда
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,одежда
...,...,...
1737,"Спасибо,подошло по размеру.Все понравилось в п...",нет товара
1738,"доставка быстрая, до Саратова около 2 недель. ...",нет товара
1739,"на внешний вид шапка нормальная, на большой об...",одежда
1740,За 4 месяца товар так и не дошел до покупателя.,нет товара


In [94]:
dataset_dishes['target'] = dataset_dishes['target'].apply(lambda x: x if x == "нет товара" else "посуда")


In [95]:
dataset_dishes.to_csv('/kaggle/working/dataset_dishes.csv')

In [96]:
!curl -T dataset_dishes.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 113 758 bytes

wget https://bashupload.com/xjlKU/taTc9.csv




In [97]:
list_of_datasets = [
    X_train_pred,
    dataset_dishes,
    dataset_house_apps,
    dataset_electronics,
    dataset_textile,
    dataset_decorations,
    dataset_shoes,
    dataset_children
]

combined_dataset = pd.concat(list_of_datasets, ignore_index=True)
df_balanced = combined_dataset.sample(frac=1).reset_index(drop=True)

In [98]:
df_balanced.to_csv('df_balanced.csv')

In [99]:
!curl -T df_balanced.csv https://bashupload.com/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Uploaded 1 file, 949 592 bytes

wget https://bashupload.com/Ntd84/df_balanced.csv




In [3]:
import pandas as pd

In [11]:
dataset_dishes = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_dishes.csv')
dataset_children = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_children.csv')
dataset_decorations = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_decorations.csv')
dataset_electronics = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_electronics.csv')
dataset_house_apps = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_house_apps.csv')
dataset_shoes = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_shoes.csv')
dataset_textile = pd.read_csv('/kaggle/input/traint-bank-sirius/dataset_textile.csv')

# Удаляем строки, где target == "нет товара" (не особо нужны, да и в датасете не очень хорошие получились)
dataset_dishes = dataset_dishes[dataset_dishes['target'] != "нет товара"]
dataset_children = dataset_children[dataset_children['target'] != "нет товара"]
dataset_decorations = dataset_decorations[dataset_decorations['target'] != "нет товара"]
dataset_electronics = dataset_electronics[dataset_electronics['target'] != "нет товара"]
dataset_house_apps = dataset_house_apps[dataset_house_apps['target'] != "нет товара"]
dataset_shoes = dataset_shoes[dataset_shoes['target'] != "нет товара"]
dataset_textile = dataset_textile[dataset_textile['target'] != "нет товара"]

list_of_datasets = [
    X_train_pred,
    dataset_dishes,
    dataset_house_apps,
    dataset_electronics,
    dataset_textile,
    dataset_decorations,
    dataset_shoes,
    dataset_children
]

combined_dataset = pd.concat(list_of_datasets, ignore_index=True)
df_balanced = combined_dataset.sample(frac=1).reset_index(drop=True)

In [7]:
dataset_shoes.shape #(344, 3) не так много но пойдет 

(344, 3)

In [16]:
df_balanced_train = df_balanced[['text', 'target']]
df_balanced_train.to_csv('df_balanced_train.csv')

### Выводы: в целом получились неплохие синтетические данные, есть конечно к чему стремится, но боюсь времени уже нет
Ключевые проблемы:
* Достаточно много генерируется примеров где название товара ставится в начало